In [69]:
# %pip install flair  tensorflow

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
%%local
import flair
import os
import pandas as pd
import numpy as np
from flair.data import Sentence

In [9]:
%%local
path = "s3://fnzoton-gbs/"
users = ['user_rating_0_to_1000.csv',
         'user_rating_1000_to_2000.csv',
         'user_rating_2000_to_3000.csv',
         'user_rating_3000_to_4000.csv',
         'user_rating_4000_to_5000.csv',
         'user_rating_5000_to_6000.csv',
         'user_rating_6000_to_11000.csv']

In [52]:
%%local
df_list = [pd.read_csv(f"{path}{f}") for f in users]
df_user = pd.concat(df_list)
df_user.drop(df_user[df_user["Name"] == "Rating"].index,axis=0, inplace=True)

In [53]:
%%local
user_ids = df_user["ID"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
book_ids = df_user["Name"].unique().tolist()
book2book_encoded = {x: i for i, x in enumerate(book_ids)}
book_encoded2book = {i: x for i, x in enumerate(book_ids)}
df_user["User"] = df_user["ID"].map(user2user_encoded)
df_user["Book"] = df_user["Name"].map(book2book_encoded)
num_users = len(user2user_encoded)
num_books = len(book_encoded2book)
num_users, num_books

(4118, 102744)

In [54]:
%%local
df_user = df_user.reset_index().rename({"index":"Index"},axis=1)

In [55]:
%%local
classifier = flair.models.TextClassifier.load('en-sentiment')

In [56]:
%%local
sentences = [Sentence(s) for s in df_user["Rating"]]

In [57]:
%%local
size = len(sentences)
for i in range(len(sentences)):
    classifier.predict(sentences[i])
    if (i+1) % 500 == 0:
        print(f"{i+1}/{size}")

500/2000
1000/2000
1500/2000
2000/2000


In [58]:
%%local
def get_score(sentences):
    return np.array([sentence.score if sentence.tag == "POSITIVE" else -sentence.score for sentence in sentences])     
sentiment_scores = get_score(sentences)

In [59]:
%%local
df_user["Scores"] = sentiment_scores

In [60]:
%%local
df_user.head(20)

Output()

In [66]:
%%local
df_user.to_csv(f"{path}rating_preprocessed.csv",index=False)